# Data engineer
A data engineer is responsible for designing, developing, and maintaining the infrastructure that is used for managing and processing large volumes of data. The key responsibilities of a data engineer may include:

* **Data pipeline development**: Creating data pipelines that extract, transform, and load (ETL) data from various sources into a centralized data warehouse or data lake.  
* **Data modeling**: Developing data models that enable efficient storage, retrieval, and analysis of data.  
* **Data integration**: Integrating data from various sources such as databases, APIs, and files, into a unified format.  
* **Data architecture**: Designing and implementing data architectures that support the requirements of various data stakeholders, including data analysts, data scientists, and business users.  
* **Data quality management**: Ensuring that data is accurate, complete, and consistent by implementing data validation and data cleaning processes.  
* **Performance optimization**: Tuning data processing systems and databases to optimize performance and reduce latency.  
* **Data security and governance**: Ensuring that data is secure and meets regulatory compliance requirements.  

Overall, data engineers play a critical role in ensuring that organizations can effectively store, manage, and analyze large volumes of data to drive business insights and decision-making.

## Airbnb Paris: how much can I charge? $$$

In [1]:
# importing libraries
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import requests

# defining constants
CITY = "Paris"

### Data collection
The first step of a data problem starts with ... data! Where do we get it? Can we collect it ourselves? Is it free? Is it legal?

Here we list some of the most common data sources:
* **Databases**: Databases are one of the primary sources of data for many organizations. Data can be collected from different types of databases, such as SQL databases (such as MySQL, Oracle, and SQL Server), NoSQL databases (such as MongoDB and Cassandra) etc.  
* **APIs**: Many organizations expose their data through APIs (Application Programming Interfaces). APIs can be used to collect data from various sources, including social media platforms, e-commerce sites, and financial data providers.
* **File systems**: Data can be collected from various file systems, including local file systems, network file systems, and cloud-based file systems such as Amazon S3 and Google Cloud Storage.
* **Web**: Data can also be collected by scraping web pages and extracting data from HTML pages, XML, and JSON documents.

Our data is in a .csv file that can be read from the web. To make it a bit faster we downloaded it already and are just reading it from our file system.

In [2]:
#df = pd.read_csv('https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/airbnb-listings/exports/csv?lang=en&facet=facet(name%3D%22host_verifications%22%2C%20disjunctive%3Dtrue)&facet=facet(name%3D%22amenities%22%2C%20disjunctive%3Dtrue)&facet=facet(name%3D%22features%22%2C%20disjunctive%3Dtrue)&refine=city%3A%22Paris%22&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B')
df = pd.read_csv('../data/airbnb-listings.csv', sep=";")
df.shape

/tmp/ipykernel_77343/138724641.py:2: DtypeWarning: Columns (40,82) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/airbnb-listings.csv', sep=";")


(54513, 89)

In [88]:
df.columns

Index(['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary',
       'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview',
       'Notes', 'Transit', 'Access', 'Interaction', 'House Rules',
       'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url',
       'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location',
       'Host About', 'Host Response Time', 'Host Response Rate',
       'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url',
       'Host Neighbourhood', 'Host Listings Count',
       'Host Total Listings Count', 'Host Verifications', 'Street',
       'Neighbourhood', 'Neighbourhood Cleansed',
       'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market',
       'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude',
       'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms',
       'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price',
       'Month

In [3]:
# Removing columns we do not need
df.drop(list(df.filter(regex = 'Url')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'URL')), axis = 1, inplace = True)
df.drop(list(df.filter(regex = 'Review')), axis = 1, inplace = True)

In [90]:
df.shape

(54513, 70)

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,ID,Scrape ID,Last Scraped,Name,Summary,Space,Description,Experiences Offered,Neighborhood Overview,Notes,Transit,Access,Interaction,House Rules,Host ID,Host Name,Host Since,Host Location,Host About,Host Response Time,Host Response Rate,Host Acceptance Rate,Host Neighbourhood,Host Listings Count,Host Total Listings Count,Host Verifications,Street,Neighbourhood,Neighbourhood Cleansed,Neighbourhood Group Cleansed,City,State,Zipcode,Market,Smart Location,Country Code,Country,Latitude,Longitude,Property Type,Room Type,Accommodates,Bathrooms,Bedrooms,Beds,Bed Type,Amenities,Square Feet,Price,Weekly Price,Monthly Price,Security Deposit,Cleaning Fee,Guests Included,Extra People,Minimum Nights,Maximum Nights,Calendar Updated,Has Availability,Availability 30,Availability 60,Availability 90,Availability 365,Calendar last Scraped,License,Jurisdiction Names,Cancellation Policy,Calculated host listings count,Geolocation,Features
0,7713011,20170404145355,2017-04-06,STUDIO PIGALLE W BALCONY,"In the heart of Montmarte Pigalle, this lovely...",This newly refurbished flat is ideal for a cou...,"In the heart of Montmarte Pigalle, this lovely...",none,It is central enough for you to walk your way ...,"Real globetrotters, we are real Parisians with...",The flat is ideally located in Montmartre Piga...,"Subway stations: Abbesses or Pigalle. Bus, Tax...",I am at your disposal to welcome you in the be...,U have to be respectfull of ur renting place a...,32492446,Vincent,2015-05-03,"Paris, Île-de-France, France","Hello everybody,\r\nI love to travel and meet ...",within an hour,100.0,NaN,Montmartre,7.0,7.0,"email,phone,reviews,jumio","Montmartre, Paris, Île-de-France 75018, France",Montmartre,Buttes-Montmartre,NaN,Paris,Île-de-France,75018,Paris,"Paris, France",FR,France,48.882926,2.335813,Apartment,Entire home/apt,2,1.0,0.0,1.0,Real Bed,"TV,Cable TV,Internet,Wireless Internet,Kitchen...",NaN,80.0,NaN,NaN,100.0,30.0,1,5,4,365,today,NaN,9,17,29,285,2017-04-06,NaN,Paris,strict,7,"48.88292573377615, 2.3358125139068453","Host Has Profile Pic,Host Identity Verified,Is..."
1,3335054,20170404145355,2017-04-05,Montmartre au pied du Sacré-Coeur !,"Ce studio de 22m2, rénové et équipé avec soin,...",A studio where it is very pleasant to settle i...,A studio where it is very pleasant to settle i...,none,The apartment is located rue d'Orsel just belo...,The building has a digital code and intercom. ...,The studio is easily accessible. At the foot o...,"The kitchen is equipped with a refrigerator, t...",The studio is adjacent to my apartment. I am r...,The apartment is non-smoking. Pets are not all...,2726446,Pascale,2012-06-24,"Paris, Île-de-France, France","Pascale, parisienne d'adoption, toulousaine de...",NaN,NaN,NaN,Montmartre,1.0,1.0,"email,phone,facebook,reviews,jumio","Montmartre, Paris, Île-de-France 75018, France",Montmartre,Buttes-Montmartre,NaN,Paris,Île-de-France,75018,Paris,"Paris, France",FR,France,48.883013,2.343469,Apartment,Entire home/apt,2,1.0,0.0,1.0,Pull-out Sofa,"Cable TV,Wireless Internet,Kitchen,Elevator in...",NaN,65.0,385.0,1250.0,150.0,15.0,1,0,3,1125,20 months ago,NaN,0,0,0,0,2017-04-05,NaN,Paris,moderate,1,"48.88301302713629, 2.343468625865973","Host Has Profile Pic,Host Identity Verified,Is..."
2,10066518,20170404145355,2017-04-06,LOVELY FLAT MONTMARTRE SACRE COEUR,In 2' by walk of the Sacré-Coeur and the Mouli...,Charmant appartement typiquement parisien avec...,In 2' by walk of the Sacré-Coeur and the Mouli...,none,"Situé en plein quartier Montmartre, vous serez...","Situé en plein quartier Montmartre, vous serez...",Très bien desservi par les transports en commu...,NaN,Nous serons là pour vous accueillir et vous do...,Nos amis les animaux ne sont pas autorisés dan...,30432969,Lea,2015-04-01,"Paris, Île-de-France, France","Vivant à Paris depuis nos études supérieures, ...",within a few hours,95.0,NaN,Montmartre,2.0,2.0,"email,phone,facebook,reviews,jumio","Montmartre, Paris, Île-de-France 75018, France",Montmartre,Buttes-Montmartre,

### Storing the data
We can always set up a database locally, however, nowadays cloud is a popular and easy choice for us to be able to share the database with the world. We set up the database using Amazon Web Services (and it is free! ... I hope).

![Alt text](../database.png)

In [201]:
#establishing the connection (local)
#conn = psycopg2.connect(
#   database="airbnb", user='postgres', password='password', host='127.0.0.1', port= '5432'
#)
#cursor = conn.cursor()

In [173]:
#establishing the connection (cloud)
credentials = pd.read_json("../credentials/db.json")
conn = psycopg2.connect(
   database="airbnb",
   user=credentials['airbnb']['username'],
   password=credentials['airbnb']['password'],
   host='airbnb.cqpzz3a3o6f2.eu-north-1.rds.amazonaws.com',
   port= '5432'
)
cursor = conn.cursor()

In [93]:
unique_roomtype = df["Room Type"].unique()
tmp = [(i,) for i in unique_roomtype]
execute_values(cursor,
    'INSERT INTO roomtype ("room_type_name") VALUES %s ON CONFLICT DO NOTHING',
    tmp)
conn.commit();

In [94]:
unique_neighbourhoods = df["Neighbourhood Cleansed"].unique()
tmp = [(i,) for i in unique_neighbourhoods]
execute_values(cursor,
    'INSERT INTO neighbourhood ("neighbourhood_name") VALUES %s ON CONFLICT DO NOTHING',
    tmp)
conn.commit();

In [95]:
unique_propertytype = df["Property Type"].unique()
tmp = [(i,) for i in unique_propertytype]
execute_values(cursor,
    'INSERT INTO propertytype ("property_type_name") VALUES %s ON CONFLICT DO NOTHING',
    tmp)
conn.commit();

In [96]:
unique_bedtype = df["Bed Type"].unique()
tmp = [(i,) for i in unique_bedtype]
execute_values(cursor,
    'INSERT INTO bedtype ("bed_type_name") VALUES %s ON CONFLICT DO NOTHING',
    tmp)
conn.commit();

In [97]:
unique_cancelpolicy = df["Cancellation Policy"].unique()
tmp = [(i,) for i in unique_cancelpolicy]
execute_values(cursor,
    'INSERT INTO cancelpolicy ("cancel_policy_name") VALUES %s ON CONFLICT DO NOTHING',
    tmp)
conn.commit();

Get additional data from an API.

In [98]:
api_url = "https://nominatim.openstreetmap.org/search?q=paris&format=json"
headers =  {"Content-Type":"application/json"}
response = requests.post(api_url, headers=headers)
response_paris = response.json()[0]
center_lon, center_lat = response_paris["lon"], response_paris["lat"]
execute_values(cursor,
    'INSERT INTO city ("city_name", "center_longitude", "center_latitude") VALUES %s ON CONFLICT DO NOTHING',
    [(CITY, center_lon, center_lat)])
conn.commit();

In [5]:
# only keep the columns that we need for the project
df = df[["ID",
        "Neighbourhood Cleansed",
        "Property Type",
        "Room Type",
        "Accommodates",
        "Bathrooms",
        "Bedrooms",
        "Beds",
        "Bed Type",
        "Price",
        "Minimum Nights",
        "Cancellation Policy",
        "Features",
        "Amenities",
        "Longitude",
        "Latitude"]]

# rename columns
df = df.rename(columns={'ID': 'id', 'Neighbourhood Cleansed': 'neighbourhood', 'Property Type': 'property_type', 'Room Type': 'room_type',
                        'Accommodates': 'accommodates', 'Bathrooms': 'bathrooms', 'Bedrooms': 'bedrooms',
                        'Beds': 'beds', 'Bed Type': 'bed_type', 'Price': 'price', 'Minimum Nights': 'minimum_nights',
                        'Cancellation Policy': 'cancel_policy', 'Features': 'features', 'Amenities': 'amenities',
                        'Longitude': 'longitude', 'Latitude': 'latitude'})

# add the city
df['city'] = CITY

In [19]:
# send raw to the database

# Postgres does not support NaN in integer columns, so we will set them to -1 (ugly)
# numeric_columns = df.select_dtypes(include='number').columns
numeric_columns = ["accommodates", "bathrooms", "bedrooms", "beds", "minimum_nights", "price"]
df[numeric_columns] = df[numeric_columns].fillna(-1).astype(int)


execute_values(
    cur=cursor,
    sql="""
        INSERT INTO raw
        (id, neighbourhood, room_type, property_type, accommodates, bathrooms, bedrooms, beds, bed_type, price,
        minimum_nights, cancel_policy, features, amenities, longitude, latitude, city)
        VALUES %s ON CONFLICT DO NOTHING;
        """,
    argslist=df.to_dict(orient="records"),
    template="""
        (
            %(id)s, %(neighbourhood)s, %(room_type)s, %(property_type)s, %(accommodates)s,
            %(bathrooms)s, %(bedrooms)s, %(beds)s, %(bed_type)s, %(price)s,
            %(minimum_nights)s, %(cancel_policy)s, %(features)s, %(amenities)s,
            %(longitude)s, %(latitude)s, %(city)s
        )
        """
    )
conn.commit()

In [20]:
# execute stored procedure to fill up the listings table
cursor.execute("CALL storelisting();")
conn.commit()

In [174]:
# get data for ds part:
ds_dat = pd.read_sql('SELECT * FROM vw_airbnb', con=conn)


/var/folders/3p/gn2n5z4x20z535hj27lp56yr0000gn/T/ipykernel_7485/2855278387.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_dat = pd.read_sql('SELECT * FROM vw_airbnb', con=conn)


In [22]:
ds_dat

,city_id,room_type_name,neighbourhood_name,longitude,latitude,price,minimum_nights,listing_given_id,property_type_name,accommodates,bathrooms,bedrooms,beds,bed_type_name,cancel_policy_name,features,amenities
0,4,Entire home/apt,Buttes-Montmartre,2.335813,48.882926,$80.00,4,7713011,Apartment,2,1,0,1,Real Bed,strict,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Kitchen..."
1,4,Entire home/apt,Buttes-Montmartre,2.343469,48.883013,$65.00,3,3335054,Apartment,2,1,0,1,Pull-out Sofa,moderate,"Host Has Profile Pic,Host Identity Verified,Is...","Cable TV,Wireless Internet,Kitchen,Elevator in..."
2,4,Entire home/apt,Buttes-Montmartre,2.340604,48.884043,$41.00,2,10066518,Apartment,2,1,1,1,Real Bed,moderate,"Host Has Profile Pic,Host Identity Verified,Is...","Internet,Wireless Internet,Kitchen,Indoor fire..."
3,4,Private room,Buttes-Montmartre,2.345101,48.885195,$50.00,2,8100080,Apartment,2,1,1,1,Real Bed,flexible,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Internet,Wireless Internet,Kitchen,Smoking ..."
4,4,Entire home/apt,Buttes-Montmartre,2.330055,48.894800,$60.00,2,7397323,Apartment,2,1,0,1,Pull-out Sofa,flexible,Host Has Profile Pic,"TV,Internet,Wireless Internet,Kitchen,Elevator..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54508,4,Entire home/apt,Palais-Bourbon,2.306002,48.858121,$70.00,2,2610162,Apartment,2,1,1,2,Real Bed,moderate,"Host Has Profile Pic,Host Identity Verified,Is...","TV,Cable TV,Internet,Wireless Internet,Kitchen..."
54509,4,Entire home/apt,Palais-Bourbon,2.325696,48.852896,$400.00,2,13760682,Apartment,5,2,2,3,Real Bed,strict,"Host Has Profile Pic,Host Identity Verified,Is...","Wireless Internet,Kitchen,Elevator in building..."
54510,4,Shared room,Palais-Bourbon,2.324841,48.852811,$0.00,1,9173969,Apartment,1,0,1,1,Real Bed,flexible,"Host Has Profile Pic,Is Location Exact","Cable TV,Carbon monoxide detector"
54511,4,Entire home/apt,Palais-Bourbon,2.316800,48.858549,$99.00,4,13754942,Apartment,3,1,2,3,Real Bed,strict,"Host Has Profile Pic,Is Location Exact","TV,Cable TV,Internet,Wireless Internet,Kitchen..."


In [175]:
conn.close()